# Importation des modules

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from ast import literal_eval
import csv
import re

In [2]:
RefNom = pd.read_csv("./../Tables/ReferentielNoms.csv",converters={"FormesNonNormalisées": literal_eval}).set_index("NomNormalisé")
RefPrenom = pd.read_csv("./../Tables/ReferentielPrenoms.csv", converters={"FormesNonNormalisées": literal_eval}).set_index("PrénomNormalisé")

In [3]:
def Parsing(CheminRelatifFichier, DivCC):
    soup = BeautifulSoup(open(CheminRelatifFichier, 'r'))
    
    #On commence par itérer à travers les balises div typées 'date'.
    
    for DivDate in soup.find_all('div', attrs={'type' : 'date'}):
        
        #On crée une variable où l'on enregistre la valeur de la date.
        
        Année = DivDate['n']
        
        
        #On crée notre Liste qui correspondra au conseil et qu'on transformera plus tard en DataFrame.
        
        ListeDicosPersonnes = []
        
        #On itère à travers les personnes.
        
        for persname in DivDate.find_all('persname'):
            
            
            DicoPersonne = {}
            
            #On commence par récupérer les informations présentes dans les balises parentes.
            
            for div in persname.find_parents('div'):
                
                if div['type'] == 'ordre' :
                    
                    DicoPersonne['corps civique'] = div['n']
                
                if div['type'] == 'corporation':
                    
                    DicoPersonne['corporation'] = div['n']
                    
                if div['type'] == 'magistrature':
                    
                    DicoPersonne['magistrature'] = div['n']
                    DicoPersonne['AnnéeMagistrature'] = Année
                
                if div['type'] == 'poele':
                    
                    DicoPersonne['poele'] = div['n']
            
            for surname in persname.find_all('surname'):
                
                DicoPersonne['nomNN'] = surname.get_text()
                
                DataToComp = surname.get_text()
                DataToComp = re.sub(r"\n", " ", DataToComp)
                if type(DataToComp) == float:
                    DicoPersonne['nom'] = ''
                for j in range (0, len(RefNom.index)):
                    CheckN = RefNom.iloc[j,1]
                    for element in CheckN:
                        if DataToComp == element:
                            DicoPersonne['nom'] = RefNom.index[j]
                            
            # Loop pour prénom non normalisé.
            
            for forename in persname.find_all('forename'):
                DicoPersonne['prénomNN'] = forename.get_text()
        
                # Loop pour prénom normalisé.
                DataToComp = forename.get_text()
                DataToComp = re.sub(r"\n", " ", DataToComp)
                if type(DataToComp) == float:
                    DicoPersonne['prénom'] = ''
                for j in range (0, len(RefPrenom.index)):
                    CheckN = RefPrenom.iloc[j,2]
                    for element in CheckN:
                        if DataToComp == element:
                            DicoPersonne['prénom'] = RefPrenom.index[j]
                            
            for addName in persname.find_all('addname'):
                DicoPersonne['surnomNN'] = addName.get_text()
                
                DataToComp = addName.get_text()
                DataToComp = re.sub(r"\n", " ", DataToComp)
                if type(DataToComp) == float:
                    DicoPersonne['surnom'] = ''
                for j in range (0, len(RefNom.index)):
                    CheckN = RefNom.iloc[j,1]
                    for element in CheckN:
                        if DataToComp == element:
                            DicoPersonne['surnom'] = RefNom.index[j]
                            
            Roles = []
            for rolename in persname.find_all('rolename'):
                Role = rolename.get_text()
                Roles.append(Role)
                DicoPersonne['Titres'] = Roles
                            
            for genname in persname.find_all('genname'):
                DicoPersonne['genName'] = genname.get_text()
                
            NomCompletNN =  persname.get_text()
            NomCompletNN = re.sub(r"\n", " ", NomCompletNN)
            DicoPersonne['NomCompletNN'] = NomCompletNN
            
            ListeDicosPersonnes.append(DicoPersonne)
            
            DfListe = pd.DataFrame(ListeDicosPersonnes)
            DfListe = DfListe.reindex(columns=['magistrature', 'AnnéeMagistrature', 'prénom', 'nom', 'surnom', 'genName', 'Titres', 'corps civique', 'corporation', 'poele', 'prénomNN', 'nomNN', 'surnomNN', 'NomCompletNN'])
            DfListe.fillna('NaN')
            DfListe.to_csv("./../Tables/ListesConseil/" + Année + "ListeConseil.csv")        
        

In [4]:
Parsing('./../Sources/EditionsXML/aves_4R_p131-220.xml', 'TEI')